# PROGRAMMING FOR DATA ANALYSIS
##  Project 2018
### DATE DUE ON/BEFORE 14 DEC 2018

Problem statement

It is requested to investigate any relationship between the touchdown distance of the aircraft known as the X-15 and the following parameters, Indicated airspeed, groundspeed and Distance to touchdown at 1 mile to landing.

It is envisaged that as a result of energy in the aircraft, that the landing distance will vary.

Data is supplied in the document 'STATISTICAL ANALYSIS OF LANDING CONTACT CONDITIONS OF THE X-15 AIRPLANE' at the URL https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19670005606.pdf 

### Delayed start due to not knowing what to pick that would not be work related and breach confidentiality at all and personal/work-related reasons

The following paramters are known to be related to the Slideout(rollout) distance.
Vertical velocity (
Indicated Airspeed at Landing
True Groundspeed at Landing


It has been found from landing data that the parameters are 
True Groundspeed = Centred on 200 knots Normal, SD = 13 knots.
Indicated Airspeed = 190 knots Normal, SD = 13 knots.
Vertical Velocity = Average 2 ft/sec, Poisson
Slideout (Rollout) distance  =  Centred on 6000 feet Normal, SD = 1000 feet
